In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split,GroupKFold,StratifiedKFold,KFold
import random
import gc
from keras.utils import to_categorical
import os
from pathlib import Path
import tensorflow as tf
from keras import backend as K
from keras.layers import Dense
from keras.models import Sequential
#import efficientnet.keras  as enf
from keras.callbacks import ReduceLROnPlateau
import keras
from keras import backend as K
from keras.models import Model, Input
from keras.layers import Dense, Lambda,GlobalAveragePooling2D
from math import ceil
import tensorflow as tf

In [ ]:
train=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
test=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv')
submission=pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/sample_submission.csv')
train_1=np.load('/kaggle/input/stratifiedgroup5folds/train_idx_fold_1.npy')
train_2=np.load('/kaggle/input/stratifiedgroup5folds/train_idx_fold_2.npy')
train_3=np.load('/kaggle/input/stratifiedgroup5folds/train_idx_fold_3.npy')
train_4=np.load('/kaggle/input/stratifiedgroup5folds/train_idx_fold_4.npy')
train_5=np.load('/kaggle/input/stratifiedgroup5folds/train_idx_fold_5.npy')
val_1=np.load('/kaggle/input/stratifiedgroup5folds/val_idx_fold_1.npy')
val_2=np.load('/kaggle/input/stratifiedgroup5folds/val_idx_fold_2.npy')
val_3=np.load('/kaggle/input/stratifiedgroup5folds/val_idx_fold_3.npy')
val_4=np.load('/kaggle/input/stratifiedgroup5folds/val_idx_fold_4.npy')
val_5=np.load('/kaggle/input/stratifiedgroup5folds/val_idx_fold_5.npy')

In [ ]:
seed_value= 42
def seed_everything(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
  # for later versions: 
    tf.compat.v1.set_random_seed(seed)
    return('Set seed {}'.format(seed))

seed_everything(seed_value)


In [ ]:
train=pd.DataFrame(train)
Y=train.target

In [ ]:
'''
train.head(5)
gkf=StratifiedKFold(n_splits=5)
train.loc[:,'Fold']=0
for fold_no,(train_index,val_index) in  enumerate(gkf.split(X=train.index,y=train['target'])):
    train.loc[train.iloc[val_index].index, 'Fold'] = fold_no
fold=0
train_0_index=train[train['Fold']!=fold].index
val_0_index=train[train['Fold']==fold].index
'''

In [ ]:
''' Resizing images
img_size=224
def preprocess_img(path,dsize=img_size):
    i=Image.open(path)
    i=i.resize((dsize,)*2,resample=Image.LANCZOS)
    return i
target=[]
n=train.shape[0]
tr_df=np.empty((n,img_size,img_size,3),dtype=np.uint8)
for i,img in enumerate(tqdm(train.image_name)):
    tr_df[i,:,:,:]=preprocess_img(f'../input/siim-isic-melanoma-classification/jpeg/train/{img}.jpg')
for i,img in enumerate(tqdm(train.image_name)):
    path=os.path.join(train_dir,img+str('.jpg'))
    try:
        c=Image.open(path)
        new_c=c.resize((img_size,)*2,resample=Image.LANCZOS)
        tr_df[i,:,:,:]=new_c
        target.append(train.target[train.image_name==img])
        
    except Exception:
        pass
np.save('x_train',tr_df)
'''

In [ ]:
resized_path='/kaggle/input/siimisic-melanoma-resized-images/'
x_train=np.load(os.path.join(resized_path,'x_train_32.npy'))/255.0
test_file=np.load(os.path.join(resized_path,'x_test_32.npy'))/255.0


In [ ]:
img_size=32

In [ ]:
tr_list_1=[train_1,train_2,train_3]
val_list_1=[val_1,val_2,val_3]
fold_index_1=[1,2,3]
tr_list_2=[train_4,train_5]
val_list_2=[val_4,val_5]
fold_index_2=[4,5]
folds_1_3=list(zip(tr_list_1,val_list_1,fold_index_1))
folds_4_5=list(zip(tr_list_2,val_list_2,fold_index_2))
#folds


In [ ]:
#x_tr,x_ts,y_tr,y_ts=train_test_split(x_train,Y,stratify=Y,shuffle=True,test_size=0.2)

In [ ]:
#!pip install '../input/kerasefficientnetb3/efficientnet-1.0.0-py3-none-any.whl'
!pip install -q efficientnet
import efficientnet.keras as efn

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
#import efficientnet.keras  as enf
from keras.callbacks import ReduceLROnPlateau
import keras
from keras import backend as K
from keras.models import Model, Input
from keras.layers import Dense, Lambda
from math import ceil
import tensorflow as tf

In [ ]:
'''
train_1_data=x_train[train_1]
val_1_data=x_train[val_1]
train_1_label=Y[train_1]
val_1_label=Y[val_1]
'''

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0, # Randomly zoom image 
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  

#datagen.fit(train_1_data)

In [ ]:
'''
import pickel as pickle
x_pk=open('xpickle.pickle','wb')
pickle.dump(X,x_pk)
x_pk.close()
y_pk=open('ypickle.pikle','wb')
pickle.dump(Y,y_pk)
y_pk.close()
x_in=open('xpickle.pickle','rb')
train_x=pickle.load(x_in)
y_in=open('ypickle.pickle','rb')
train_y=pickle.load(y_in)
'''

In [ ]:
import tensorflow as tf
gm_exp = tf.Variable(3.0, dtype = tf.float32)
def generalized_mean_pool_2d(X):
    #import tensorflow
    #gm_exp = tf.Variable(3.0, dtype = tf.float32)
    #import tensorflow as tf
    pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                        axis = [1, 2], 
                        keepdims = False) + 1.e-7)**(1./gm_exp)
    return pool

In [ ]:
dense_1=256
dense_2=128

In [ ]:
def create_model(input_shape,dense_1=dense_1,dense_2=dense_2):
    # Input Layer
    input = Input(shape = input_shape)
    
    # Create and Compile Model and show Summary
    x_model = efn.EfficientNetB7(weights = 'noisy-student', include_top = False, input_tensor = input, pooling = None, 
                                 classes = None)
    
    # UnFreeze all layers
    for layer in x_model.layers:
        layer.trainable = True
    
    # GeM
    lambda_layer = Lambda(generalized_mean_pool_2d)
    lambda_layer.trainable_weights.extend([gm_exp])
    
    x = lambda_layer(x_model.output)
     # multi output
    #x=Dense(dense_1,activation='relu')(x)
    x=Dense(dense_2,activation='relu')(x)
    classes= Dense(1, activation = 'sigmoid', name = 'class')(x)
    

    # model
    model = Model(inputs = x_model.input, outputs = [classes])
    return model

In [ ]:
gc.collect()
#del model3 mode


In [ ]:
HEIGHT_NEW=WIDTH_NEW=img_size
CHANNELS=3

model2 = create_model(input_shape = (HEIGHT_NEW, WIDTH_NEW, CHANNELS))

In [ ]:
#model_name='B7x32_256128_0.h5'

In [ ]:
from tensorflow.keras import backend as K
import dill
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

In [ ]:
'''
reduce_learning_rate = ReduceLROnPlateau(monitor='loss',factor=0.2,patience=2,cooldown=2,min_lr=0.00001,verbose=1)
callbacks = [reduce_learning_rate]
model0.compile( optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model0.fit_generator(datagen.flow(train_0_data,train_0_label,batch_size=32),validation_data=(val_0_data,val_0_label),
                     epochs=50,callbacks=callbacks)
#model3.fit(x_tr,y_tr,batch_size=32,epochs=50,callbacks=callbacks)
model0.save(model_name)
'''

In [ ]:
'''
gc.collect()
del train_1
del train_1_data
del val_1
del val_1_data
del train_1_label
del val_1_label
'''

In [ ]:
'''
# for single fold only module
reduce_learning_rate = ReduceLROnPlateau(monitor='loss',factor=0.2,patience=2,cooldown=2,min_lr=0.00001,verbose=1)
callbacks = [reduce_learning_rate]
model2.compile( optimizer='adam',loss=[binary_focal_loss(alpha=.25, gamma=2)],metrics=[tf.keras.metrics.AUC()])
model2.fit_generator(datagen.flow(train_1_data,train_1_label,batch_size=64),alidation_data=(val_1_data,val_1_label)
        ,epochs=50,callbacks=callbacks,workers=-1,use_multiprocessing=True)
'''

In [ ]:
gc.collect()

In [ ]:
test['target']=0
reduce_learning_rate = ReduceLROnPlateau(monitor='val_auc',factor=0.2,patience=2,cooldown=2,min_lr=0.00001,verbose=1)
callbacks = [reduce_learning_rate]
model2.compile( optimizer='adam',loss=[binary_focal_loss(alpha=.25, gamma=2)],metrics=[tf.keras.metrics.AUC()])
for (tr_ind,val_ind,fold_ind) in folds_1_3:
    X_train=x_train[tr_ind]
    Y_train=Y[tr_ind]
    X_val=x_train[val_ind]
    Y_val=Y[val_ind]
    datagen.fit(X_train)
    print('Training for fold{}'.format(fold_ind))
    model2.fit_generator(datagen.flow(X_train,Y_train,batch_size=64),validation_data=(X_val,Y_val)
        ,epochs=50,callbacks=callbacks,workers=-1,use_multiprocessing=True)
    pred = model2.predict(test_file)
    #print(labels[:4])
    prediction = [i[0]/5  for i in pred]
    test['target']+=prediction


In [ ]:
model_name='B7X32n_sgkfd2_gem_64_.h5'
model2.save(model_name)
ensemble=pd.DataFrame({'image_name': test.image_name})
ensemble['target']=test.target
ensemble.to_csv('ensembled{}_1_3fold.csv'.format(model_name))


In [ ]:
model=[model3]
def comare_model(models):
    output=[]
    names=['effecientnet','vgg16','resnet']
    for mod in models:
        res=mod.evaluate(x_ts,y_ts)
        roc=roc_auc_score(y_ts,mod.predict(x_ts))
        output.append((res[0],res[1],roc))
    output=pd.DataFrame(output,index=names,columns=['loss','accuracy','roc_score'])
    return(output)
result=comare_model(model)
result.to_csv(model_name+str('.csv'))
result
        

In [ ]:
#roc_auc_score(y_ts,model3.predict(x_ts))
model_name='B7X32n_sgkfd2_2gem_64_.h5'


In [ ]:
from keras.models import load_model
import tensorflow as tf
#model3=load_model('/kaggle/input/model-first/32.h5',custom_objects={'tf': tf,'gm_exp':gm_exp})

In [ ]:
from datetime import datetime
import pytz
tz_ist=pytz.timezone('Asia/Kolkata')
dtnow=datetime.now(tz_ist)
import time
timestr = dtnow.strftime("%Y%m%d-%H%M")
#print timestr
#op
#inverse={'0':'Food', '1':'misc', '2':'Attire', '3':'Decorationandsignage'}
#inverse={'0':'manipuri', '1':'bharatanatyam', '2':'odissi', '3':'kathakali', '4':'kathak','5':'sattriya','6':'kuchipudi',
 #        '7':'mohiniyattam'}
labels = model2.predict(test_file)
#print(labels[:4])
l1 = [i[0]  for i in labels]
#class_label = [inverse[str(x)] for x in label]
#print(class_label[:3])
submission = pd.DataFrame({ 'image_name': test.image_name, 'target': ensemble.tarhet })
submission.head(10)
submission.to_csv('submission{0}{1}.csv'.format(model_name,timestr), index=False)

In [ ]:
ensemble=pd.DataFrame({'image_name': test.image_name})
ensemble['fold1']=0
ensemble['fold2']=l1
ensemble['fold3']=0
ensemble['fold4']=0
ensemble['fold5']=0
ensemble['total']=ensemble['fold1']+ensemble['fold2']+ensemble['fold3']+ensemble['fold4']+ensemble['fold5']
ensemble['mean']=ensemble['total']/5
ensemble['weighted']=0.2*ensemble['fold1']+0.2*ensemble['fold2']+0.2*ensemble['fold3']+0.2*ensemble['fold4']+0.2*ensemble['fold5']
ensemble.to_csv('ensembled{}.csv'.format(model_name))

In [ ]:
ensemble

In [ ]:
img_size=32
def preprocess_img(path,dsize=img_size):
    i=Image.open(path)
    i=i.resize((dsize,)*2,resample=Image.LANCZOS)
    return i

n=test.shape[0]
ts_df=np.empty((n,img_size,img_size,3),dtype=np.uint8)
for i,img in enumerate(tqdm(test.image_name)):
    ts_df[i,:,:,:]=preprocess_img(f'../input/siim-isic-melanoma-classification/jpeg/test/{img}.jpg')